In [1]:
from collections import namedtuple
import bs4 as BeautifulSoup
import urllib.request

In [2]:
url = 'https://sf.eater.com/maps/best-coffee-shops-san-francisco-oakland-berkeley'

In [3]:
data = urllib.request.urlopen(url)
read_data = data.read()

In [4]:
parsed_data = BeautifulSoup.BeautifulSoup(read_data,'html.parser')

In [5]:
titles = list(map(
        lambda x: x.text[1:],
        parsed_data.select('div.c-mapstack__cards--improved section h1')[1:]
        ))
paragraphs = list(map(lambda x: x.text, filter(lambda y: 'id=' not in str(y), 
             parsed_data.select('div.c-mapstack__cards--improved section div.c-entry-content p'))))

In [6]:
import pandas as pd
import re

titles = [re.sub(r'[0-9]+. ', '', t) for t in titles]

corpus_df = pd.DataFrame({'Name': titles, 
                          'Description': paragraphs})
corpus_df = corpus_df[['Name', 'Description']]

In [7]:
"""
from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
all credits go to alko and arturomp @ stack overflow.
"""

import re
cList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "I'm": "I am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

# examples
#print expandContractions('Don\'t you get it?')

In [8]:
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer 
from nltk.tokenize import word_tokenize, sent_tokenize

#nltk.download('punkt')
#nltk.download('wordnet')

wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

new_words=('yeah', 'yea')
for i in new_words:
    stop_words.append(i)
    
stemmer = PorterStemmer() 
    
def normalizer(paragrahp):
    
    paragrahp = paragrahp.lower()
    paragrahp = expandContractions(paragrahp)
    paragrahp = paragrahp.split('.')
    paragrahp = paragrahp[:-1]
    # lower case and remove special character/whitespace
    paragrahp = [re.sub(r'[^a-zA-Z\s]','', sentence, re.I|re.A) for sentence in paragrahp]
    #paragrahp = [sentence.lower() for sentence in paragrahp]
    paragrahp = [sentence.strip() for sentence in paragrahp]
    tokens = [wpt.tokenize(sentence) for sentence in paragrahp]
    tokens_filtered = [[word for word in token if word not in stop_words] for token in tokens]
    print (tokens_filtered)
    tokens_stemmed = [[stemmer.stem(word) for word in token] for token in  tokens_filtered]
    print (tokens_stemmed)
    doc = [' '.join(tokens) for tokens in tokens_stemmed]
    return doc

norm_corpus = normalizer(corpus_df['Description'][0])

[['got', 'toast', 'crave'], ['cinnamon', 'sugar', 'slathered', 'nutella', 'mitigate', 'toast', 'cravings'], ['also', 'got', 'best', 'coffee', 'neighborhood'], ['super', 'tiny', 'inside', 'expect', 'find', 'seat', 'instead', 'head', 'best', 'parklet', 'town', 'composed', 'huge', 'twisted', 'log'], ['also', 'great', 'place', 'seek', 'refuge', 'hours', 'long', 'wait', 'brunch', 'outerlands', 'next', 'door']]
[['got', 'toast', 'crave'], ['cinnamon', 'sugar', 'slather', 'nutella', 'mitig', 'toast', 'crave'], ['also', 'got', 'best', 'coffe', 'neighborhood'], ['super', 'tini', 'insid', 'expect', 'find', 'seat', 'instead', 'head', 'best', 'parklet', 'town', 'compos', 'huge', 'twist', 'log'], ['also', 'great', 'place', 'seek', 'refug', 'hour', 'long', 'wait', 'brunch', 'outerland', 'next', 'door']]


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True, ngram_range=(2,2))
tv_matrix = tv.fit_transform(norm_corpus)
tv_matrix = tv_matrix.toarray()

vocab = tv.get_feature_names()
pd.DataFrame(np.round(tv_matrix, 2), columns=vocab)

,also got,also great,best coffe,best parklet,brunch outerland,cinnamon sugar,coffe neighborhood,compos huge,expect find,find seat,...,seat instead,seek refug,slather nutella,sugar slather,super tini,tini insid,toast crave,town compos,twist log,wait brunch
0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.00,0.00,...,0.00,0.0,0.00,0.00,0.00,0.00,0.63,0.00,0.00,0.0
1,0.0,0.0,0.0,0.00,0.0,0.42,0.0,0.00,0.00,0.00,...,0.00,0.0,0.42,0.42,0.00,0.00,0.34,0.00,0.00,0.0
2,0.5,0.0,0.5,0.00,0.0,0.00,0.5,0.00,0.00,0.00,...,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
3,0.0,0.0,0.0,0.27,0.0,0.00,0.0,0.27,0.27,0.27,...,0.27,0.0,0.00,0.00,0.27,0.27,0.00,0.27,0.27,0.0
4,0.0,0.3,0.0,0.00,0.3,0.00,0.0,0.00,0.00,0.00,...,0.00,0.3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.3
